In [152]:
import pandas as pd
import numpy as np

In [153]:
df = pd.read_csv('download.csv')

In [154]:
df['3day_tokenprice_mean'] = df['PRICE'].rolling(3).mean()
df['3day_liq_amount_mean'] = df['LIQUIDATION_AMOUNT_USD'].rolling(3).mean()
df['3day_tokenprice_std'] = df['PRICE'].rolling(3).std()
df['3day_liq_amount_std'] = df['LIQUIDATION_AMOUNT_USD'].rolling(3).std()





In [155]:
df["3day_price_shift"] = df['PRICE'] - df['PRICE'].shift(3)
df["3day_liq_amount_shift"] = df['LIQUIDATION_AMOUNT_USD'] - df['LIQUIDATION_AMOUNT_USD'].shift(3)


In [156]:
df['1std_significant_price_downtrend'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['3day_price_shift'] > (-df['3day_tokenprice_std']), 0)
df['1std_significant_liquidation_downtrend'] = pd.DataFrame(np.ones((len(df['3day_liq_amount_shift']) -1))).where(df['3day_price_shift'] < (df['3day_liq_amount_std']), 0)
df['5std_significant_price_downtrend'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['3day_price_shift'] > (-5 * df['3day_tokenprice_std']), 0)
df['5std_significant_liquidation_downtrend'] = pd.DataFrame(np.ones((len(df['3day_liq_amount_shift']) -1))).where(df['3day_price_shift'] < (5 * df['3day_liq_amount_std']), 0)
df['10std_significant_price_downtrend'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['3day_price_shift'] > (-10 * df['3day_tokenprice_std']), 0)
df['10std_significant_liquidation_downtrend'] = pd.DataFrame(np.ones((len(df['3day_liq_amount_shift']) -1))).where(df['3day_price_shift'] < (10 * df['3day_liq_amount_std']), 0)

In [157]:
df['1_correlated_significant_events'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['1std_significant_price_downtrend'] + df['1std_significant_liquidation_downtrend'] != 2, 0)
df['5_correlated_significant_events'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['5std_significant_price_downtrend'] + df['5std_significant_liquidation_downtrend'] != 2, 0)
df['10_correlated_significant_events'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['10std_significant_price_downtrend'] + df['10std_significant_liquidation_downtrend'] != 2, 0)

In [165]:
print("Number of correlated events vs. individual events for 1 std:  \n")
print("Number of price and liquidation events: ", df['1_correlated_significant_events'].sum(), "\n")
print("Number of price events: ",  df['1std_significant_price_downtrend'].sum(), "\n" )
print("Number of liquidation events: ", df['1std_significant_liquidation_downtrend'].sum(), "\n") 
print("Number of correlated events vs. individual events for 5 std:  \n")
print("Number of price and liquidation events: ", df['5_correlated_significant_events'].sum(), "\n")
print("Number of price events: ",  df['5std_significant_price_downtrend'].sum(), "\n" )
print("Number of liquidation events: ", df['5std_significant_liquidation_downtrend'].sum(), "\n") 
print("Number of correlated events vs. individual events for 10 std:  \n")
print("Number of price and liquidation events: ", df['10_correlated_significant_events'].sum(), "\n")
print("Number of price events: ",  df['10std_significant_price_downtrend'].sum(), "\n" )
print("Number of liquidation events: ", df['10std_significant_liquidation_downtrend'].sum(), "\n") 

Number of correlated events vs. individual events for 1 std:  

Number of price and liquidation events:  104.0 

Number of price events:  157.0 

Number of liquidation events:  258.0 

Number of correlated events vs. individual events for 5 std:  

Number of price and liquidation events:  18.0 

Number of price events:  243.0 

Number of liquidation events:  258.0 

Number of correlated events vs. individual events for 10 std:  

Number of price and liquidation events:  12.0 

Number of price events:  249.0 

Number of liquidation events:  258.0 



In [160]:
def avg_recovery(price, price_shift, sig_events):
  recovery_sum = 0
  recovery_events = 0
  for i in range(len(price)):
    if sig_events[i] == 1:
      j = i
      if j == len(price):
          return recovery_sum / recovery_events
      while price[j] - price[i] < abs(price_shift[i]):
        j = j + 1
        if j == len(price):
          return recovery_sum / recovery_events
      recovery_sum += j - i
      recovery_events += 1
  return recovery_sum / recovery_events




  

In [167]:
avg_price_recovery_25 = avg_recovery(list(df['PRICE']), list(df['3day_price_shift']), list(df['1_correlated_significant_events']))
avg_price_recovery_50 = avg_recovery(list(df['PRICE']), list(df['3day_price_shift']), list(df['5_correlated_significant_events']))
avg_price_recovery_75 = avg_recovery(list(df['PRICE']), list(df['3day_price_shift']), list(df['10_correlated_significant_events']))
print("Average recovery time for a 1 std deviation correlated event", avg_price_recovery_25)
print("Average recovery time for a 5 std deviation correlated event", avg_price_recovery_50)
print("Average recovery time for a 10 std deviation correlated event", avg_price_recovery_75)

Average recovery time for a 1 std deviation correlated event 17.37
Average recovery time for a 5 std deviation correlated event 14.833333333333334
Average recovery time for a 10 std deviation correlated event 13.75


In [166]:
df

,DATE,LIQUIDATION_AMOUNT,LIQUIDATION_AMOUNT_USD,BORROWER_VOLUME,PRICE,3day_tokenprice_mean,3day_liq_amount_mean,3day_tokenprice_std,3day_liq_amount_std,3day_price_shift,3day_liq_amount_shift,1std_significant_price_downtrend,1std_significant_liquidation_downtrend,5std_significant_price_downtrend,5std_significant_liquidation_downtrend,10std_significant_price_downtrend,10std_significant_liquidation_downtrend,1_correlated_significant_events,5_correlated_significant_events,10_correlated_significant_events
0,2021-05-22 00:00:00.000,2.480861e+05,1.745014e+06,1347,7.393356,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,2021-05-23 00:00:00.000,1.935383e+06,1.020416e+07,4508,5.351693,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,2021-05-24 00:00:00.000,2.876607e+04,1.403412e+05,138,5.908958,6.218002,4.029838e+06,1.055333,5.406979e+06,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,2021-05-25 00:00:00.000,9.115849e+03,5.351080e+04,38,6.234085,5.831579,3.466004e+06,0.446256,5.835575e+06,-1.159270,-1.691504e+06,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,2021-05-26 00:00:00.000,3.001614e+02,2.119543e+03,7,7.145935,6.429659,6.532386e+04,0.641261,6.986395e+04,1.794242,-1.020204e+07,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2022-04-07 00:00:00.000,3.918696e+04,4.026553e+06,97,105.849085,111.598404,1.631685e+06,5.342783,2.117172e+06,-7.594612,3.916954e+06,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
258,2022-04-08 00:00:00.000,9.415367e+04,9.045356e+06,181,100.859770,106.414771,4.643816e+06,5.858363,4.127670e+06,-15.550901,9.036395e+06,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
259,2022-04-09 00:00:00.000,3.222899e+04,2.996101e+06,35,94.560294,100.423049,5.356003e+06,5.657053,3.236348e+06,-17.975164,2.136561e+06,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
260,2022-04-11 00:00:00.000,2.582589e+05,2.285039e+07,353,86.804845,94.074970,1.163062e+07,7.040020,1.017649e+07,-19.044239,1.882384e+07,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
